## Imports

In [1]:
from IPython.display import Math
from lib3d_mec_ginac import *
from math import pi, e

In [2]:
g = get_parameter('g')

In [3]:
 # Integration step
delta_t =             .003
# Assembly init problem solver parameters
geom_eq_init_tol =    1e-10
geom_eq_init_relax =  .1
# Assembly problem solver parameters
geom_eq_tol =         delta_t**2 * 10**-3
geom_eq_relax =       .1
# Equilibrium problem solver parameters
dyn_eq_tol =          1.0e-10
dyn_eq_relax =        .1
# Perturbed dynamic state solver parameters
per_dyn_state_tol =   1e-12
# ...

Generalized coordinates, velocities and accelerations

In [4]:
theta1, dtheta1, ddtheta1 = new_coordinate('theta1', -pi/6, 0)
theta2, dtheta2, ddtheta2 = new_coordinate('theta2', -2*pi/6, 0)
theta3, dtheta3, ddtheta3 = new_coordinate('theta3', -3*pi/6, 0)

In [5]:
get_coordinates()

theta1       -0.5236     
theta2       -1.0472     
theta3       -1.5708     

In [6]:
get_velocities()

dtheta1      0.0         
dtheta2      0.0         
dtheta3      0.0         

Geometric parameters

In [7]:
l1, l2 = new_parameter('l1', 0.4), new_parameter('l2', 2.0)
l3, l4 = new_parameter('l3', 1.2), new_parameter('l4', 1.6)

In [8]:
#new_base('Barm1', 'xyz', [0, 1, 0], theta1)
#new_base('Barm2', 'xyz', [0, 1, 0], theta2)
#new_base('Barm3', 'xyz', [0, 1, 0], theta3)

TypeError: 

In [ ]:
get_bases()